Creating the data-set:

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [3]:
def create_data(direc):
    img_path=[]
    label=[]
    for i in os.listdir(direc): # for each directory (angry, happy etc) in directory
        for img in os.listdir(os.path.join(direc,i)): # for each img in the directory 
            img_path.append(os.path.join(direc,i,img)) 
            label.append(i)
    return img_path, label


In [4]:
train_image, train_label = create_data("images/train")
train_data = {"images": train_image, "labels": train_label}
train = pd.DataFrame(train_data)

train.head()

,images,labels
0,images/train\angry\0.jpg,angry
1,images/train\angry\1.jpg,angry
2,images/train\angry\10.jpg,angry
3,images/train\angry\10002.jpg,angry
4,images/train\angry\10016.jpg,angry


In [5]:
val_image, val_label = create_data("images/validation")
val_data = {"images": val_image, "labels": val_label}
val = pd.DataFrame(val_data)

val.head()

,images,labels
0,images/validation\angry\10052.jpg,angry
1,images/validation\angry\10065.jpg,angry
2,images/validation\angry\10079.jpg,angry
3,images/validation\angry\10095.jpg,angry
4,images/validation\angry\10121.jpg,angry


In [6]:
from keras_preprocessing.image import load_img
def features(images):
    f = []
    for img in tqdm(images):
        img = load_img(img, grayscale=True)
        img = np.array(img)
        f.append(img)

    f = np.array(f)
    f = f.reshape(len(f), 48,48,1) # n, length, breadth, depth
    return f


train_features = features(train["images"])
val_features = features(val["images"])



  0%|          | 0/28821 [00:00<?, ?it/s]c:\Users\Hemakshi Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 7066/7066 [01:58<00:00, 59.86it/s] 


In [7]:
x_train = train_features/255.0
x_val = val_features/255.0

In [8]:
from sklearn.preprocessing import LabelEncoder

labelencoder= LabelEncoder()
labelencoder.fit(train["labels"])

LabelEncoder()

In [9]:
y_train = labelencoder.transform(train["labels"])
y_val = labelencoder.transform(val["labels"])

In [10]:
import keras_preprocessing
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes = 7)
y_val = to_categorical(y_val, num_classes=7)

y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

Training the model:

In [11]:
from keras.models import Sequential
from keras import layers

model = Sequential([
    # Input: (48, 48, 1)
    layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),  # (48 → 24)
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D((2, 2)),  # (24 → 12)
    layers.Dropout(0.3),

    layers.Conv2D(256, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D((2, 2)),  # (12 → 6)
    layers.Dropout(0.3),

    # Upsample back from 6 → 12
    layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu'),  # 6 → 12
    layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu'),   # 12 → 24
    layers.Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu'),   # 24 → 48

    # Fixed output layers for classification
    layers.GlobalAveragePooling2D(),  # Reduces (48, 48, 32) to (32,)
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')  # Output: (None, 7) for 7 emotion classes
])


c:\Users\Hemakshi Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.compile(optimizer = "adam", loss="categorical_crossentropy", metrics = ["accuracy"])
model.fit(x = x_train, y = y_train, batch_size= 128, epochs = 40, verbose = 0, validation_data=(x_val,y_val))

In [20]:
model_json = model.to_json()
with open("facial-emotion-detector.json","w") as f:
    f.write(model_json)
model.save("facial-emotion-detector.h5")

In [21]:
emotion_label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [22]:


def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [23]:
image = 'images/train/angry/188.jpg'
print("Original image: angry")
img = ef(image)
pred = model.predict(img)
print(pred.argmax())
pred_label = emotion_label[pred.argmax()]
print("Model prediction:",pred_label)

Original image: angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
0
Model prediction: angry


In [24]:
val_loss, val_accuracy = model.evaluate(x_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Accuracy: 0.6118
